In [ ]:
pip install opencv-python opencv-contrib-python

In [ ]:
import cv2

# Load the dictionary that was used to generate the markers.
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()

# Start webcam
cap = cv2.VideoCapture(0)

print("Press 'q' to quit the webcam window.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect markers in the image
    corners, ids, rejected = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

    # Draw markers if detected
    if ids is not None:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        for i, marker_id in enumerate(ids):
            print(f"Detected marker ID: {marker_id[0]}")

    # Show the image
    cv2.imshow("ArUco Marker Detection", frame)

    # Quit with 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything
cap.release()
cv2.destroyAllWindows()


Distance Test V1

In [ ]:
import cv2
import numpy as np

# Load your calibration parameters here
# You need to replace these with actual values from your camera calibration
camera_matrix = np.array([[800, 0, 320],
                          [0, 800, 240],
                          [0, 0, 1]], dtype=float)
dist_coeffs = np.zeros((4, 1))  # Assume no distortion for simplicity (adjust if needed)

# Marker settings
marker_length = 0.05  # In meters (5 cm)

# Load dictionary
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect markers
    corners, ids, _ = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

    if ids is not None and len(ids) >= 2:
        # Estimate pose of each marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)

        # Draw markers and axes
        for i in range(len(ids)):
            cv2.aruco.drawDetectedMarkers(frame, corners)
            cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.03)

        # Measure distance between first two detected markers
        tvec1 = tvecs[0][0]
        tvec2 = tvecs[1][0]

        distance = np.linalg.norm(tvec1 - tvec2)
        text = f"Distance: {distance*100:.2f} cm"
        cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

    # Show the frame
    cv2.imshow("Distance Between Markers", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Distance Test (+ averaging)

In [ ]:
import cv2
import numpy as np
from collections import deque

# Load your calibration parameters here
camera_matrix = np.array([[800, 0, 320],
                          [0, 800, 240],
                          [0, 0, 1]], dtype=float)
dist_coeffs = np.zeros((4, 1))  # Assume no distortion for simplicity

# Marker settings
marker_length = 0.05  # 5 cm markers

# Load ArUco dictionary and detector
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()

# Initialize webcam
cap = cv2.VideoCapture(0)

# Store last 20 distances
last_distances = deque(maxlen=20)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect ArUco markers
    corners, ids, _ = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

    if ids is not None and len(ids) >= 2:
        # Estimate pose for each marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(
            corners, marker_length, camera_matrix, dist_coeffs
        )

        # Draw detected markers and their axes
        for i in range(len(ids)):
            cv2.aruco.drawDetectedMarkers(frame, corners)
            cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.03)

        # Measure distance between first two markers
        tvec1 = tvecs[0][0]
        tvec2 = tvecs[1][0]
        distance = np.linalg.norm(tvec1 - tvec2)  # In meters

        # Add to the last_distances buffer
        last_distances.append(distance)

        # Display live distance
        cv2.putText(frame, f"Distance: {distance*100:.2f} cm",
                    (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

    # Display average over the last 20 frames
    if last_distances:
        avg_distance = sum(last_distances) / len(last_distances)
        cv2.putText(frame, f"Avg (Last 20): {avg_distance*100:.2f} cm",
                    (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)

    # Show the video frame
    cv2.imshow("Distance Between Markers", frame)

    # Quit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Distance Test (+averaging +calibration1)

In [6]:
import cv2
import numpy as np
from collections import deque

data = np.load('camera_calibration.npz')
camera_matrix = data['camera_matrix']
dist_coeffs = data['dist_coeffs']

# Marker settings
marker_length = 0.032  # 5 cm markers

# Load ArUco dictionary and detector
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()

# Initialize webcam
cap = cv2.VideoCapture(0)

# Store last 20 distances
last_distances = deque(maxlen=20)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect ArUco markers
    corners, ids, _ = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

    if ids is not None and len(ids) >= 2:
        # Estimate pose for each marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(
            corners, marker_length, camera_matrix, dist_coeffs
        )

        # Draw detected markers and their axes
        for i in range(len(ids)):
            cv2.aruco.drawDetectedMarkers(frame, corners)
            cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.03)

        # Measure distance between first two markers
        tvec1 = tvecs[0][0]
        tvec2 = tvecs[1][0]
        distance = np.linalg.norm(tvec1 - tvec2)  # In meters

        # Add to the last_distances buffer
        last_distances.append(distance)

        # Display live distance
        cv2.putText(frame, f"Distance: {distance*100:.2f} cm",
                    (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

    # Display average over the last 20 frames
    if last_distances:
        avg_distance = sum(last_distances) / len(last_distances)
        cv2.putText(frame, f"Avg (Last 20): {avg_distance*100:.2f} cm",
                    (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)

    # Show the video frame
    cv2.imshow("Distance Between Markers", frame)

    # Quit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Deadline 24
Goal = แอปกระสือ ยังใช้จริงไม่ได้

mark = หาสมการ + ตัวเลข + workflow (มี citation จะดีมาก)
speed = python integration
indy, poon = pages

deadline app = 18


